In [11]:
import simpy
import numpy.random as random
import pandas as pd
import scipy.stats as st
import heapq
from heapq import heappush
from collections import namedtuple

def id_generator():
    """Generate unique, global customer ID."""
    i = 1
    while True:
        yield i
        i += 1

class IATDistribution:
    """
    Help control the table of Inter-Arrival Time Distribution table.
    """
    def __init__(self, path):
        self.df = pd.read_csv(path)
        self.df["Floor"] = self.df["Floor"].apply(lambda x:x.lstrip('0'))
        
    def getter(self, location, section, direction, floor):
        
        # filter row by mulitple conditions         
        temp = self.df[(self.df['Location'] == location) & (self.df['Section'] == section) & \
                       (self.df['Direction'] == direction) & (self.df['Floor'] == floor)][['Distribution','Parameters']]
        
        # return distribution name and parsed parameters
        return {
            'dist': getattr(self.st, temp['Distribution'].values[0]),
            'params': self.params_parser(temp['Parameters'].values[0])
        }
    
    @staticmethod
    def params_parser(params_string):
        params = []
        for param in params_string.replace("(", "").replace(")", "").split(", "): 
            params.append(float(param))
        return params

In [2]:


Customer = namedtuple("Customer", ["cid", "direction", "destination", "time"])
Mission = namedtuple("Mission", ["direction", "destination"])


class AssignmentError(Exception):
    """Exception : Fail to assign a elevator."""
    pass
        
def heaptop(heap):
    return heap[0][1]
def heappop(heap):
    item = heapq.heappop(heap)
    return item[1]
    
class Queue:
    def __init__(self, env, floor, direction):
        self.env = env
        self.floor = floor
        self.direction = direction
        self.queue_array = []
        self.arrival_event = self.env.event()
        self.inflow_proc = self.env.process(self.inflow())
        self.outflow_proc = self.env.process(self.outflow())
        
    def inflow(self):
        while True:
            customers = yield self.arrival_event
            print('OuterCalls from Floor',self.floor,' / Direction','up' if self.direction == 1 else 'down', self.env.now)
            
            if(len(self.queue_array) == 0):
                global CALL_EVENT
                mission = Mission(direction=self.direction, destination=self.floor)
                CALL_EVENT.succeed(value=mission)   
                CALL_EVENT = self.env.event()
                
            self.queue_array = self.queue_array + customers  
                          
    def outflow(self):
        while True :
            capacity, occupationNum, elevIndex = yield AAA_EVENT[self.direction][self.floor]
                   
            riders = []
                       
            vacancy = capacity-occupationNum
            while (vacancy > 0) and (self.queue_array):
                riders.append(self.queue_array.pop())
                vacancy -= 1
            yield self.env.timeout(len(riders) * random.randint(WALKING_MIN, WALKING_MAX))
                
            print(vacancy,' people on board','elev:', elevIndex)
                
            BBB_EVENT[elevIndex].succeed(value=riders)
            BBB_EVENT[elevIndex] = self.env.event()
    
class Floor:
    def __init__(self, env, floor, direction, IAT, DD):
        self.env = env
        self.floor = floor
        self.direction = direction
        self.dist = IAT['dist']
        self.params = IAT['params']
        self.queue = Queue(env, self.floor, self.direction)
        self.source_proc = env.process(self.Source(env))
        
        self.DD = DD
        if direction == 1: 
            self.DD = self.DD[self.floor.lstrip("0"):]
        elif direction == -1:
            self.DD = self.DD[:self.floor.lstrip("0")]
        if len(self.DD) == 1:
            pass
        else:
            self.DD /= self.DD.sum()
        
    def Source(self, env):
        while True:
            t = -1
            while t < 0:
                t = self.dist.rvs(*self.params[:-2], loc = self.params[-2], scale = self.params[-1], size = 1)
            yield self.env.timeout(t)     

            customers = []
            for i in range(random.randint(1,3)):
                to = random.choice(self.DD.index, p=self.DD)
                customers.append(Customer(cid=next(id_gen), direction=self.direction, destination=to, time=self.env.now))

            self.queue.arrival_event.succeed(value = customers)
            self.queue.arrival_event = self.env.event()


class Elevator:
    def __init__(self, env, elevIndex, failAssignment):
        self.env = env
        self.elevIndex = elevIndex
        self.current_floor = '1'
        self.capacity = 15
        self.riders = []
        self.stop_list = []
        self.assign_event = self.env.event()
        self.direction = 0
        self.env.process(self.idle())
        self.failAssignment = failAssignment
        
    def resubmit(self):
        for i in range(len(self.failAssignment)):
            mission = self.failAssignment.pop()
            global RESUBMIT_EVENT
            RESUBMIT_EVENT.succeed(value=mission)
            RESUBMIT_EVENT = self.env.event()
            
    def idle(self):
        while True:            
            print('Elevator:',self.elevIndex, 'idle')
            mission = yield self.assign_event
            index = self.floorPriority(mission.destination, self.direction)
            heappush(self.stop_list,(index,mission))

            if self.distansebetween(self.current_floor, mission.destination) > 0:
                self.direction = 1
            elif self.distansebetween(self.current_floor, mission.destination) < 0:
                self.direction = -1
            else:
                self.direction = mission.direction    
            
            print('Elev {} Moving to Floor {} / Direction {}'.format(self.elevIndex,mission.destination,'up' if self.direction == 1 else 'down'))
            yield self.env.process(self.onMission())
            
            self.direction = 0
            self.resubmit()
    
    def onMission(self):
        while self.stop_list != []:
            print('Elev',self.elevIndex,'Stoplist:\n  ',self.stop_list)
            moving_proc = self.env.process(self.moving( heaptop(self.stop_list).destination, self.current_floor))
            
            while self.current_floor != heaptop(self.stop_list).destination:
                    
                mission = yield self.assign_event | moving_proc
                    
                if self.assign_event.triggered:
                    print('New Assigenment : ', mission)
                    
                    before = heaptop(self.stop_list).destination
                    index = self.floorPriority(mission.destination, self.direction)
                    heappush(self.stop_list, (index, mission ))
    
                    if before != heaptop(self.stop_list).destination:
                        print('Change Piriority.')
                        moving_proc.interrupt()
                        moving_proc = self.env.process(self.moving( heaptop(self.stop_list).destination, self.current_floor))
            
            print('Start serving at', self.current_floor)
            yield self.env.process(self.serving())
        
    def travelingTime(self, destination, source):
        return 10
    def moving(self, destination, source):
        print('Start Moving')
        try:   
            while self.current_floor != destination:
                t = self.travelingTime(destination, source)
                yield self.env.timeout(t)
                self.current_floor = self.forwards(self.current_floor, self.direction)
                print('Update Floor to', self.current_floor)
                print(self.stop_list)
        except simpy.Interrupt as i:
            print('Change destination')
            
    def serving(self):
        
        # customers leaving
        for index, rider in enumerate(self.riders):
            if rider.destination == self.current_floor:
                print('Customer leave')
                self.riders.pop(index)
        
        mission = heappop(self.stop_list)
        
        if (self.current_floor == FloorList[-1] and  self.direction == 1) or \
        (self.current_floor == FloorList[0] and self.direction == -1):
            return
        
        AAA_EVENT[mission.direction][self.current_floor].succeed( value = (self.capacity, len(self.riders), self.elevIndex))
        AAA_EVENT[mission.direction][self.current_floor] = self.env.event()

        print("Elev {} serve Floor {}".format(self.elevIndex, self.current_floor))
                                                                       
        riders = yield BBB_EVENT[self.elevIndex]
        print('Riders', riders,)
        for customer in riders:
            if(customer.destination not in [t[1].destination for t in self.stop_list]):
                
                mission = Mission(direction=customer.direction, destination=customer.destination)
                index = self.floorPriority(mission.destination, self.direction)
                print('InnerCalls:',mission, 'at Floor', self.current_floor)
                heappush(self.stop_list, (index, mission))
        print('StopList', self.stop_list)
        self.riders = self.riders + riders
    
    @staticmethod
    def distansebetween(floor1, floor2):
        floor1 = int(floor1) if not 'B' in floor1 else -int(floor1[1:])+1 
        floor2 = int(floor2) if not 'B' in floor2 else -int(floor2[1:])+1
        return floor2-floor1  
    
    @staticmethod
    def floorPriority(floor, direction):
        index = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        if direction == -1:
            index = -index
        return index  
    
    @staticmethod
    def forwards(floor, direction):
        floor = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        floor += direction
        floor = str(floor) if floor > 0 else "B{}".format(abs(floor-1))
        return floor
        
class ElevatorController:
    def __init__(self, env, elevatorList):
        self.env = env
        self.elevatorList = elevatorList        
        self.failAssignment = []
        self.elevators = dict()
        for elevatorName in elevatorList:
            self.elevators[elevatorName] = Elevator(env, elevatorName, self.failAssignment)
        self.env.process(self.assignCalls())
    
    def assignCalls(self):
        while True:
            mission = yield CALL_EVENT | RESUBMIT_EVENT
            for i in mission:
                mission = mission[i]
            print('[{}] {}'.format('Call' if CALL_EVENT.triggered else 'Resubmit', mission))
            try:
                candidate = self.bestCandidate(mission.direction, mission.destination)
                
                self.elevators[candidate].assign_event.succeed(value=mission)
                self.elevators[candidate].assign_event = self.env.event()
            except AssignmentError:
                print('Fail to Assign')
                
    def bestCandidate(self, direction, source):
        """Assignment Policy"""
        minDistance = 50
        bestElevator = str()
        for elevator in self.elevators.values():
            if (minDistance > self.distansebetween(source, elevator.current_floor) and \
               ((direction == elevator.direction and elevator.current_floor*direction < direction*source) \
                or elevator.direction == 0)):
                    minDistance = self.distansebetween(source, elevator.current_floor)
                    bestElevator = elevator.elevIndex
            
        if minDistance == 50:
            self.failAssignment.append(Mission(direction=direction, destination=source))
            raise AssignmentError()
            
        return bestElevator
    
    @staticmethod
    def distansebetween(floor1, floor2):
        floor1 = int(floor1) if not 'B' in floor1 else -int(floor1[1:])+1 
        floor2 = int(floor2) if not 'B' in floor2 else -int(floor2[1:])+1
        return abs(floor2-floor1)

In [12]:
random.seed(111)
     
# Inter-Arrival Time Distribution
IATD = IATDistribution('./BestFitDistribution - 複製.csv')

# Destination Distribution
DD = pd.read_csv('./FloorRatio_NHB.csv').iloc[:,1:].set_index('from').iloc[4:8,4:8]

# Location name
Location = "北棟病床"

# Time Section
TimeSection = 2

# Floor List
FloorList = ['1','2', '3', '4']

# Elevator ID
ElevatorList = ['3001']

# Enviornment Variable
env = simpy.Environment()
id_gen = id_generator()
WALKING_MAX = 2
WALKING_MIN = 1

# Global Event
CALL_EVENT = env.event()
RESUBMIT_EVENT = env.event()
AAA_EVENT = { 1: { i : env.event() for i in FloorList}, -1: { i : env.event() for i in FloorList} }
BBB_EVENT = { i: env.event() for i in ElevatorList }

# process
floors_upward   = [ Floor(env, i,  1, IATD.getter(Location, TimeSection,   'up', i), DD.loc[i]) for i in FloorList[ :-1]]
floors_downward = [ Floor(env, i, -1, IATD.getter(Location, TimeSection, 'down', i), DD.loc[i]) for i in FloorList[1:  ]]
elevator_ctrl = ElevatorController(env, ElevatorList)

env.run(until=100)

Elevator: 3001 idle
OuterCalls from Floor 4  / Direction down [25.27901171]
[Resubmit] Mission(direction=-1, destination='4')
Elev 3001 Moving to Floor 4 / Direction up
Elev 3001 Stoplist:
   [(4, Mission(direction=-1, destination='4'))]
Start Moving
Update Floor to 2
[(4, Mission(direction=-1, destination='4'))]
Update Floor to 3
[(4, Mission(direction=-1, destination='4'))]
OuterCalls from Floor 1  / Direction up [54.73201577]
[Resubmit] Mission(direction=1, destination='1')
Fail to Assign
Update Floor to 4
[(4, Mission(direction=-1, destination='4'))]
Start serving at 4
Elevator: 3001 idle
[Resubmit] Mission(direction=1, destination='1')
Elev 3001 Moving to Floor 1 / Direction down
Elev 3001 Stoplist:
   [(1, Mission(direction=1, destination='1'))]
Start Moving
Update Floor to 3
[(1, Mission(direction=1, destination='1'))]
Update Floor to 2
[(1, Mission(direction=1, destination='1'))]
Update Floor to 1
[(1, Mission(direction=1, destination='1'))]
Start serving at 1
Elevator: 3001 id